#ADMINISTRAÇÃO FINANCEIRA DE ACADEMIAS, SEJAM ELAS de EDUCAÇÃO FÍSICA, SEJAM PARA EXERCÍCIOS DE PILATES

#Autor do programa e escolha do tema

# Sistema: Adm. Financeira de Academias e ambientes de preparação física
# Locais: Acondicionamento físico e qualidade de vida das pessoas

In [70]:
#---------------------------
# Caregando bibliotecas
#---------------------------
import os
import pandas as pd
import unicodedata, re # formatação de arquivos para utf-8


In [71]:
#-----------------------------------------------------------------
# Formatando variáveis de saída com valores em duas casas decimais
#-----------------------------------------------------------------
pd.options.display.float_format = "{:,.2f}".format

In [72]:
# =============================================================================
# Primeira rotina: Montagem de Drive somente se estiver no Google Colab
# =============================================================================
def em_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False

if em_colab():
    from google.colab import drive
    drive.mount('/content/drive')

# =============================================================================
# Segunda rotina: Helpers de leitura e de diretório
# =============================================================================
def garantir_dir(caminho_arquivo: str):
    pasta = os.path.dirname(caminho_arquivo)
    if pasta and not os.path.exists(pasta):
        os.makedirs(pasta, exist_ok=True)

def read_excel_smart(path_or_url: str) -> pd.DataFrame:
    """
    Leitor 'esperto' de Excel:
    - Para .xls tenta engine='xlrd' (pandas exige xlrd instalado para .xls)
    - Para .xlsx deixa o pandas escolher (openpyxl)
    - Mensagem amigável se xlrd não estiver instalado.
    """
    # is_xls = path_or_url.lower().endswith(".xls")
    # try:
    #     if is_xls:
    #         return pd.read_excel(path_or_url, engine="xlrd")
    #     else:
    #         return pd.read_excel(path_or_url)  # .xlsx, .xlsm etc.
    # except ImportError as e:
    #     if is_xls and "xlrd" in str(e).lower():
    #         raise RuntimeError(
    #             "Para ler .xls é necessário instalar o pacote 'xlrd'. "
    #             "No Colab: !pip install xlrd\nNo Anaconda/Local: pip install xlrd"
    #         ) from e
    #     raise
    # except Exception as e:
    #     # fallback simples: deixe a exceção aparecer com contexto
    #     raise
    try:
        # Try reading as CSV first, as the remote files seem to be in this format
        df = pd.read_csv(path_or_url)
        return df
    except Exception as e:
        # If reading as CSV fails, fall back to the original Excel reading logic
        is_xls = path_or_url.lower().endswith(".xls")
        try:
            if is_xls:
                return pd.read_excel(path_or_url, engine="xlrd")
            else:
                return pd.read_excel(path_or_url)  # .xlsx, .xlsm etc.
        except ImportError as e_excel:
            if is_xls and "xlrd" in str(e_excel).lower():
                raise RuntimeError(
                    "Para ler .xls é necessário instalar o pacote 'xlrd'. "
                    "No Colab: !pip install xlrd\nNo Anaconda/Local: pip install xlrd"
                ) from e_excel
            raise e_excel
        except Exception as e_excel:
            # fallback simples: deixe a exceção aparecer com contexto
            raise e_excel

# =======================================================================================
# Terceira rotina: Leitura dos arquivos do GitHub e salvamento no Drive (quando houver)
# =======================================================================================
URL_PLANOS = "https://raw.githubusercontent.com/Jvanderlei1954/lab_meus_testes_cursos_2025/main/planos_acad.xls"
URL_CUSTOS = "https://raw.githubusercontent.com/Jvanderlei1954/lab_meus_testes_cursos_2025/main/custos_acad.xls"
URL_SOCIOS = "https://raw.githubusercontent.com/Jvanderlei1954/lab_meus_testes_cursos_2025/main/socios_acad.xls"

df_planos = read_excel_smart(URL_PLANOS)
df_custos = read_excel_smart(URL_CUSTOS)
df_socios = read_excel_smart(URL_SOCIOS)

# Add debug print to check column names after reading
#print("\n--- Columns after reading ---")
#print("df_planos columns:", df_planos.columns.tolist())
#print("df_custos columns:", df_custos.columns.tolist())
#print("df_socios columns:", df_socios.columns.tolist())
#print("-----------------------------")


# Caminhos de saída (se estiver fora do Colab, você pode trocar por um caminho local)
SAIDA1 = "/content/drive/MyDrive/MeuCurso/planos2_acad.csv"
SAIDA2 = "/content/drive/MyDrive/MeuCurso/custos2_acad.csv"
SAIDA3 = "/content/drive/MyDrive/MeuCurso/socios2_acad.csv"

for caminho in (SAIDA1, SAIDA2, SAIDA3):
    garantir_dir(caminho)

#==============================================================================
# Quarta rotina: Geração de réplicas dos arquivos já tratados no padrão utf-8
#==============================================================================
df_planos.to_csv(SAIDA1, index=False, encoding="utf-8-sig")
print(f"Arquivo salvo em: {SAIDA1}")

df_custos.to_csv(SAIDA2, index=False, encoding="utf-8-sig")
print(f"Arquivo salvo em: {SAIDA2}")

df_socios.to_csv(SAIDA3, index=False, encoding="utf-8-sig")
print(f"Arquivo salvo em: {SAIDA3}")

# =============================================================================
# Sexta rotina:Normalização e mapeamento de colunas
# =============================================================================
def _normalize_token(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # remove acento
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # só letras/números
    return s

def _build_mapper(groups: dict) -> dict:
    m = {}
    for dest, syns in groups.items():
        all_syns = list(syns) + [dest]  # inclui o próprio canônico
        for s in all_syns:
            m[_normalize_token(s)] = dest
    return m
#************************************************************************
# Sétima rotina: Rearrumando as colunas para o padrão de leitura
#************************************************************************
def _fix_joined_header(df, expected_cols):
    """
    Se o DF tem só 1 coluna e o nome dela contém vírgulas (ex.: 'Plano,Preco,Alunos'),
    divide o conteúdo por vírgula/; e renomeia para expected_cols.
    """
    if set(expected_cols).issubset(df.columns):
        return df  # já está ok

    if len(df.columns) == 1:
        col0 = df.columns[0]
        # Caso 1: o NOME da coluna já tem as vírgulas (ex.: 'Plano,Preco,Alunos')
        if ("," in col0) or (";" in col0):
            parts = df[col0].astype(str).str.split(r"[;,]\s*", expand=True)
            # se vier com mais colunas do que o esperado, corta
            parts = parts.iloc[:, :len(expected_cols)]
            parts.columns = expected_cols
            return parts

        # Caso 2: o nome da coluna é algo genérico, mas as LINHAS têm valores separados por vírgula
        sample_has_sep = df[col0].astype(str).head(10).str.contains(r"[;,]").any()
        if sample_has_sep:
            parts = df[col0].astype(str).str.split(r"[;,]\s*", expand=True)
            parts = parts.iloc[:, :len(expected_cols)]
            parts.columns = expected_cols
            return parts

    # Se nada funcionou, retorna como veio (deixando a verificação bater depois)
    return df

# ---- Use assim, antes da canonicalização ----
REQUIRED_PLANOS = ["Plano", "Preco", "Alunos"]
df_planos = _fix_joined_header(df_planos, REQUIRED_PLANOS)

# Garanta tipos numéricos depois de “desgrudar”
if set(REQUIRED_PLANOS).issubset(df_planos.columns):
    df_planos["Preco"]  = pd.to_numeric(df_planos["Preco"], errors="coerce").fillna(0.0)
    df_planos["Alunos"] = pd.to_numeric(df_planos["Alunos"], errors="coerce").fillna(0).astype(int)

# Agora sua rotina deve rodar sem o KeyError:
df_planos = canonicalize_columns(df_planos, PLANOS_GROUPS, REQUIRED_PLANOS, "planos", debug=True)

def canonicalize_columns(df: pd.DataFrame,
                         groups: dict,
                         required: list,
                         where: str,
                         debug: bool=False) -> pd.DataFrame:
    synmap = _build_mapper(groups)

    # rename preservando original se não mapeado
    new_cols = {}
    for c in df.columns:
        tok = _normalize_token(c)
        new_cols[c] = synmap.get(tok, c.strip())

    df2 = df.rename(columns=new_cols)

    # Resolver colisões (duplicatas após rename)
    counts = {}
    fixed_names = []
    for col in df2.columns:
        counts[col] = counts.get(col, 0) + 1
        if counts[col] == 1:
            fixed_names.append(col)
        else:
            fixed_names.append(f"{col}__dup{counts[col]-1}")
    if fixed_names != list(df2.columns):
        df2.columns = fixed_names

    #if debug:
       #print(f"\n[{where}] === DEBUG ===")
       #print("Colunas originais:", list(df.columns))
       #print("Rename preview:", dict(list(new_cols.items())[:10]))
       #print("Colunas finais:", list(df2.columns))

    # checagem de obrigatórios
    missing = [r for r in required if r not in df2.columns]
    if missing:
        dica = (
            "Verifique se os nomes de 'required' batem com as chaves de 'groups' "
            "e se os sinônimos cobrem variações (acentos, espaços, etc.)."
        )
        raise KeyError(
            f"[{where}] Colunas obrigatórias ausentes: {missing}. "
            f"Disponíveis: {list(df2.columns)}. Dica: {dica}"
        )
    return df2

#=================================================================================
# Oitava rotina: Elaboração de dicionários para tratamento dos nomes das colunas
#=================================================================================
# ---- Grupos ----
PLANOS_GROUPS = {
    "Plano":   ["plano", "planos", "tipo", "categoria", "modalidade"],
    "Preco":   ["preco", "preço", "valor", "mensalidade", "contrato", "precoaluno", "valoraluno", "taxa", "tarifa"],
    "Alunos":  ["alunos", "qtdalunos", "qtdealunos", "qtd", "qtde", "quantidade", "aluno", "matriculados", "assinantes"],
}
CUSTOS_GROUPS = {
    "Custo": ["custo", "despesa", "item", "descricao", "descrição"],
    "Valor": ["valor", "valor(r$)", "valor_rs", "valorrs", "valorreais", "valormensal", "montante"],
}
SOCIOS_GROUPS = {
    "Socio":      ["socio", "sócio", "socia", "sócia", "nome", "parceiro"],
    "Percentual": ["percentual", "perc", "porcentagem", "%", "participacao", "participação", "quota", "cota", "fracao", "fração"],
}

# ---- Aplicar canonicalização ----
df_planos = canonicalize_columns(df_planos, PLANOS_GROUPS, ["Plano", "Preco", "Alunos"], "planos", debug=True)
df_custos = canonicalize_columns(df_custos, CUSTOS_GROUPS, ["Custo", "Valor"], "custos", debug=True)
df_socios = canonicalize_columns(df_socios, SOCIOS_GROUPS, ["Socio", "Percentual"], "socios", debug=True)

# =======================================================================================
# Nona rotina: Tipagem segura (evita AttributeError em .fillna quando coluna não existe)
# =======================================================================================
# Como os required garantem a existência, podemos converter direto:
df_planos["Preco"]  = pd.to_numeric(df_planos["Preco"], errors="coerce").fillna(0.0)
df_planos["Alunos"] = pd.to_numeric(df_planos["Alunos"], errors="coerce").fillna(0.0)

df_custos["Valor"]  = pd.to_numeric(df_custos["Valor"], errors="coerce").fillna(0.0)

# Percentual pode estar em 0-1 ou 0-100; se detectar >1, assume % e divide por 100
df_socios["Percentual"] = pd.to_numeric(df_socios["Percentual"], errors="coerce").fillna(0.0)
if (df_socios["Percentual"] > 1).any():
    df_socios["Percentual"] = df_socios["Percentual"] / 100.0

# =============================================================================
# Décima rotina: Visualização rápida os arquivos
# =============================================================================
#print("\n=== Planos (padronizado) ===")
#print(df_planos.head())
#print("\n=== Custos (padronizado) ===")
#print(df_custos.head())
#print("\n=== Sócios (padronizado) ===")
#print(df_socios.head())

# Exemplo de métricas simples
#receita_mensal_estimada = (df_planos["Preco"] * df_planos["Alunos"]).sum()
#custo_mensal_total = df_custos["Valor"].sum()
#lucro_bruto = receita_mensal_estimada - custo_mensal_total

#print(f"\nReceita mensal estimada: {receita_mensal_estimada:,.2f}")
#print(f"Custo mensal total:      {custo_mensal_total:,.2f}")
#print(f"Lucro bruto:             {lucro_bruto:,.2f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo salvo em: /content/drive/MyDrive/MeuCurso/planos2_acad.csv
Arquivo salvo em: /content/drive/MyDrive/MeuCurso/custos2_acad.csv
Arquivo salvo em: /content/drive/MyDrive/MeuCurso/socios2_acad.csv


In [73]:
#**************************************************************************************************
# Décima rotina: Tratamento das colunas criando constantes para identificação de erros de conteúdo
#**************************************************************************************************
# ---------------- 1b) tipos - tratamento das colunas ----------------
df_planos["Preco"]  = pd.to_numeric(df_planos["Preco"], errors="coerce").fillna(0.0)
df_planos["Alunos"] = pd.to_numeric(df_planos["Alunos"], errors="coerce").fillna(0).astype(int)
df_custos["Valor"]  = pd.to_numeric(df_custos["Valor"], errors="coerce").fillna(0.0)
df_socios["Percentual"] = pd.to_numeric(df_socios["Percentual"], errors="coerce").fillna(0.0)

# Percentual para prolabore dos sócios - pode vir como 50 ou 0.5
df_socios["Frac"] = df_socios["Percentual"].apply(lambda x: x/100.0 if x > 1 else x)
total_frac = df_socios["Frac"].sum()
if total_frac > 0:
    df_socios["Frac"] = df_socios["Frac"] / total_frac
else:
    # sem sócios válidos: ninguém recebe pró-labore
    df_socios["Frac"] = 0.0


In [74]:
#******************************************************************************************************
# Décima primeira rotina: Montando o plano para vendas de pacotes de seções de exercícios na acadamia
#******************************************************************************************************
# ---------------- 2) receita mensal por plano ----------------
def mensalizar(plano: str, preco: float) -> float:
    p = (plano or "").strip().lower()
    if p == "semestral":
        return preco / 6.0
    elif p == "trimestral":
        return preco / 3.0
    elif p == "anual":
        return preco / 12.0
    else:
        return preco

receita_dados = []
for _, row in df_planos.iterrows():
    plano = str(row["Plano"]).strip().title()
    preco = float(row["Preco"])
    alunos = int(row["Alunos"])
    vm_aluno = mensalizar(plano, preco)
    rec_mensal = vm_aluno * alunos
    receita_dados.append([plano, alunos, preco, vm_aluno, rec_mensal])

df_receita = pd.DataFrame(
    receita_dados,
    columns=["Plano", "Qtde Alunos", "Contrato(R$)", "Valor Mensal por Aluno (R$)", "Receita Mensal (R$)"]
)
for col in ["Contrato(R$)", "Valor Mensal por Aluno (R$)", "Receita Mensal (R$)"]:
    df_receita[col] = df_receita[col].round(2)

receita_total = float(df_receita["Receita Mensal (R$)"].sum())



In [75]:
#*******************************************************************************************************
# Décima segunda rotina: definição da rotina de cálculos para apuração dos valores a serem apresentados
#*******************************************************************************************************
# ---------------- 3) custos e tributos ----------------
custo_total = float(df_custos["Valor"].sum())
tot_tributos = round(receita_total * 0.06, 2)

# ---------------- 4) pró-labore ----------------
lucro_operacional = max(receita_total - custo_total, 0.0)
base_prolab = round(lucro_operacional * 0.40, 2)
desc_inss = round(base_prolab * 0.11, 2)
desc_irpf = round(base_prolab * 0.14, 2)
val_prolab_liq = round((base_prolab - desc_inss - desc_irpf) + (receita_total / 5.0), 2)

if df_socios["Frac"].sum() > 0:
    dist_prolab = {row["Socio"]: round(val_prolab_liq * row["Frac"], 2) for _, row in df_socios.iterrows()}
else:
    dist_prolab = {}
tot_prolab = round(sum(dist_prolab.values()), 2)

# ---------------- 5) resultados ----------------
lucro_liquido = round((receita_total - custo_total) - (tot_prolab + tot_tributos), 2)
perc_liq = (lucro_liquido / receita_total * 100.0) if receita_total else 0.0
despesa_total = round(receita_total - lucro_liquido, 2)
perc_despesa_total = (despesa_total / receita_total * 100.0) if receita_total else 0.0



In [76]:
#************************************************************************************************************
# Décima terceira rotina: Estabelecendo visões sobre os planos, valores e a temporalidade de suas aplicações
#************************************************************************************************************
# ---------------- 6) visões trimestral e anual ----------------
map_vm = {r["Plano"]: r["Valor Mensal por Aluno (R$)"] for _, r in df_receita.iterrows()}
map_qt = {r["Plano"]: r["Qtde Alunos"] for _, r in df_receita.iterrows()}
vm_mensal    = map_vm.get("Mensal", 0.0)
vm_trimestral= map_vm.get("Trimestral", 0.0)
vm_semestral = map_vm.get("Semestral", 0.0)
vm_anual     = map_vm.get("Anual", 0.0)
qa_mensal     = map_qt.get("Mensal", 0)
qa_trimestral = map_qt.get("Trimestral", 0)
qa_semestral  = map_qt.get("Semestral", 0)
qa_anual      = map_qt.get("Anual", 0)

fat_mes   = (vm_mensal     * qa_mensal)     * 3
fat_trim  = (vm_trimestral * qa_trimestral) * 3
fat_seme  = (vm_semestral  * qa_semestral)  * 3
fat_ano   = (vm_anual      * qa_anual)      * 3
total_trimestre = round(fat_mes + fat_trim + fat_seme + fat_ano, 2)

demo_mes   = (vm_mensal     * qa_mensal)     * 12
demo_trim  = (vm_trimestral * qa_trimestral) * 12
demo_seme  = (vm_semestral  * qa_semestral)  * 12
demo_ano   = (vm_anual      * qa_anual)      * 12
demo_rec_tot = round(demo_mes + demo_trim + demo_seme + demo_ano, 2)

In [77]:
#*********************************************************************************************************
# Décima quarta rotina: Apresentação dos dados financeiros apurados em função dos pacotes comercializados
#*********************************************************************************************************
# ---------------- 7) impressão ----------------
print("="*86)
print(" ACADEMIAS - RELATÓRIO FINANCEIRO RESUMIDO")
print("="*86)

print("\n--- Receita por Plano ---")
print(df_receita[["Plano","Qtde Alunos","Contrato(R$)","Valor Mensal por Aluno (R$)","Receita Mensal (R$)"]]
      .sort_values("Plano").to_string(index=False))

print(f"Consolidado mensal ====================================================== >>> {receita_total:,.2f}")

print("")
print("=" * 86)
print("ACADEMIAS - DETALHAMENTO DO RELATÓRIO FINANCEIRO")
print("=" * 86)
print(f"Mensal:     Val_Aula = {vm_mensal:,.2f}     | Trimestre = {fat_mes:,.2f}   | Fatura/Mês = {vm_mensal*qa_mensal:,.2f}")
print(f"Trimestral: Val_Aula = {vm_trimestral:,.2f}     | Trimestre = {fat_trim:,.2f}   | Fatura/Mês = {vm_trimestral*qa_trimestral:,.2f}")
print(f"Semestral:  Val_Aula = {vm_semestral:,.2f}     | Trimestre = {fat_seme:,.2f}   | Fatura/Mês =  {vm_semestral*qa_semestral:,.2f}")
print(f"Anual:      Val_Aula = {vm_anual:,.2f}     | Trimestre = {fat_ano:,.2f}   | Fatura/Mês =  {vm_anual*qa_anual:,.2f}")
print("="*86)
print(f"Fat_Trimestral: --------------------------> R$ {total_trimestre:,.2f}   | Anual    R$ {demo_rec_tot:,.2f}")
print(" ")


 ACADEMIAS - RELATÓRIO FINANCEIRO RESUMIDO

--- Receita por Plano ---
     Plano  Qtde Alunos  Contrato(R$)  Valor Mensal por Aluno (R$)  Receita Mensal (R$)
     Anual           15      4,308.00                       359.00             5,385.00
    Mensal           40        423.00                       423.00            16,920.00
 Semestral           15      2,286.00                       381.00             5,715.00
Trimestral           30      1,200.00                       400.00            12,000.00
Consolidado mensal ====================================================== >>> 40,020.00

ACADEMIAS - DETALHAMENTO DO RELATÓRIO FINANCEIRO
Mensal:     Val_Aula = 423.00     | Trimestre = 50,760.00   | Fatura/Mês = 16,920.00
Trimestral: Val_Aula = 400.00     | Trimestre = 36,000.00   | Fatura/Mês = 12,000.00
Semestral:  Val_Aula = 381.00     | Trimestre = 17,145.00   | Fatura/Mês =  5,715.00
Anual:      Val_Aula = 359.00     | Trimestre = 16,155.00   | Fatura/Mês =  5,385.00
Fat_Trimestr

In [81]:
#*******************************************************************************************
# Décima quinta rotina: Apresentação dos resultados obtidos na operação do empreendimento
#*******************************************************************************************
print("=" * 86)
print("DEMONSTRATIVO DOS RESULTADOS")
print("=" * 86)
print(f"Receita Total Mensal:            R$ {receita_total:,.2f}")
print(f"Custos Fixos (Despesas Perman.): R$ {custo_total:,.2f}")
print(f"Tributos (6% s/ receita):        R$ {tot_tributos:,.2f}")
print("\n Pró-Labore                       Rubricas")
for nome, valor in dist_prolab.items():
    print(f"{nome:<30} R$ {valor:,.2f}")
print(f"Somatório dos Pró-labores:     R${tot_prolab:,.2f}")

print("\n--- Despesas & Resultado ---")
print(f"Despesas Totais Mensais:         R$ {despesa_total:,.2f}")
print(f"Perc. Despesa Total:             {perc_despesa_total:,.2f}%")
print("="*86)
print(f">>> Lucro Líquido Mensal:        R$ {lucro_liquido:,.2f}")
print(f">>> Lucro Líquido:               {perc_liq:,.2f}% da Receita Total Mensal")
print("="*86)

DEMONSTRATIVO DOS RESULTADOS
Receita Total Mensal:            R$ 40,020.00
Custos Fixos (Despesas Perman.): R$ 6,680.00
Tributos (6% s/ receita):        R$ 2,401.20

 Pró-Labore                       Rubricas
Socio1-50%                     R$ 8,184.55
Socio2-30%                     R$ 4,910.73
Socio3-30%                     R$ 4,910.73
Somatório dos Pró-labores:     R$18,006.01

--- Despesas & Resultado ---
Despesas Totais Mensais:         R$ 27,087.21
Perc. Despesa Total:             67.68%
>>> Lucro Líquido Mensal:        R$ 12,932.79
>>> Lucro Líquido:               32.32% da Receita Total Mensal


## Visualização para análise dos arquivos de entrada

In [ ]:
#-------------------------------------------------------
# Extra: Analisando arquivos encaminhados para o Google Drive
#-------------------------------------------------------
print("\n=== Planos ===")
print(df_planos.head())

print("\n=== Custos ===")
print(df_custos.head())

print("\n=== Socios ===")
print(df_socios.head())